In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("data4.csv")

In [3]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(df)

fig = px.scatter_3d(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    z=X_pca[:, 2],
    color=df['y'],
    labels={'x': 'PC1', 'y': 'PC2', 'z': 'PC3'},
    title='PCA of data4.csv'
)
fig.show()

In [4]:

X = df.drop(columns=['y'])
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_raw = LinearRegression(fit_intercept=True)
model_raw.fit(X_train, y_train)
y_raw_pred = model_raw.predict(X_test)

mse_raw = mean_squared_error(y_test, y_raw_pred)
r2_raw = r2_score(y_test, y_raw_pred)
print(f"Raw Data - MSE: {mse_raw}, R2: {r2_raw}")

Raw Data - MSE: 232.8268236033175, R2: 0.9689854069305857


In [5]:
# with alpha=1.0

model_l2 = make_pipeline(StandardScaler(), Ridge())
model_l2.fit(X_train, y_train)

y_l2_pred = model_l2.predict(X_test)
mse_l2 = mean_squared_error(y_test, y_l2_pred)
r2_l2 = r2_score(y_test, y_l2_pred)
print(f"L2 Regularization - MSE: {mse_l2}, R2: {r2_l2}")

L2 Regularization - MSE: 229.5887052985138, R2: 0.9694167529412377


In [13]:
# ridgecv
import numpy as np

alphas = np.logspace(-5, 5, 50)

pipe_cv = make_pipeline(StandardScaler(), RidgeCV(alphas=alphas, scoring='neg_mean_squared_error', cv=5))
pipe_cv.fit(X_train, y_train)
best_alpha_cv = pipe_cv.named_steps['ridgecv'].alpha_
y_pred_cv = pipe_cv.predict(X_test)
print("RidgeCV - best alpha:", best_alpha_cv)
print("RidgeCV - MSE:", mean_squared_error(y_test, y_pred_cv), "R2:", r2_score(y_test, y_pred_cv))

RidgeCV - best alpha: 0.49417133613238384
RidgeCV - MSE: 230.77348979595263 R2: 0.969258929162627
